In [1]:
import numpy as np

In [98]:
#constructing model for forward-backward algorithm 
class forward_backward:
    def __init__(self,states,obs,observation,trans,obs_trans,bias):
        self.states = states
        self.obs = obs
        self.observation = observation
        self.trans = trans
        self.obs_trans = obs_trans
        self.bias = bias
    
    def forward(self,state,t):
        state_index = np.where(states==state)[0][0]
        if t == 0:
            return self.bias[state_index]*self.obs_trans[self.observation[0]][state_index]
        else:
            alpha = 0
            for i in range(len(self.states)):
                alpha += self.trans[state_index][i]*self.forward(self.states[i],t-1)
            return self.obs_trans[state_index][self.observation[t]]*alpha
    
    def backward(self,state,t):
        state_index = np.where(states==state)[0][0]
        if t == (len(self.observation)-1):
            return 1
        else:
            beta = 0
            for i in range(len(self.states)):
                beta += self.trans[state_index][i]*self.obs_trans[i][self.observation[t+1]]*self.backward(self.states[i],t+1)
            return beta
        
    def predict_probabilites(self):
        predictions = []
        for t in range(len(self.observation)):
            probs = []
            for i in range(len(self.states)):
                x_t = self.states[i]
                probs.append(self.obs_trans[i][self.observation[t]]*self.backward(x_t,t)*self.forward(x_t,t))
            predictions.append(probs)
        return predictions
    
    def predict_states(self):
        predictions = []
        for t in range(len(self.observation)):
            probs = []
            for i in range(len(self.states)):
                x_t = self.states[i]
                probs.append(self.obs_trans[i][self.observation[t]]*self.backward(x_t,t)*self.forward(x_t,t))
            predictions.append(self.states[np.argmax(probs)])
        return predictions

In [94]:
#constructing model for viterbi algorithm 
class viterbi:
    def __init__(self,states,obs,observation,trans,obs_trans,bias):
        self.states = states
        self.obs = obs
        self.observation = observation
        self.F = -np.log(trans)
        self.G = -np.log(obs_trans)
        self.bias = bias
        self.beta = []
    
    def backward(self,state,t):
        state_index = np.where(states==state)[0][0]
        if t == (len(self.observation)-1):
            return 0
        else:
            beta_list = []
            for i in range(len(self.states)):
                x_t = self.states[i]
                beta_list.append(self.G[i][self.observation[t+1]] + self.F[state_index][i] + self.backward(x_t,t+1))
            return min(beta_list)
    
    def construct_beta(self):
        beta_list = []
        for state_index in range(len(self.states)):
            x_t = self.states[state_index]
            beta = []
            for t in range(len(self.observation)):
                beta.append(self.backward(x_t,t))
            beta_list.append(beta)
        return beta_list
    
    def forward(self,t,beta):
        if t == 0:
            x_list = []
            for i in range(len(self.states)):
                x_list.append(self.G[i][self.observation[t]] - np.log(self.bias[i]) + beta[i][t])
            return self.states[np.argmin(x_list)]
        else:
            x_list = []
            for i in range(len(self.states)):
                x_t = self.states[i]
                x_list.append(self.G[i][self.observation[t]] - self.F[i][self.forward(t-1,beta)] + beta[i][t])
            return self.states[np.argmin(x_list)]
    
    def predict_states(self,beta):
        predictions = []
        for t in range(len(self.observation)):
            predictions.append(self.forward(t,beta))
        return predictions

In [ ]:
#defining states and probabilities
states = np.array([0,1])
obs = np.array([0,1,2])
observation = np.array([0,1,1,0,2,0,2,2,2,0,2,2,2,2,2,0,0,1,1,2])
trans = np.array([[.5,.5],[.5,.5]])
obs_trans = np.array([[.5,.4,.1],[.4,.1,.5]])
bias = np.array([.5,.5])

In [99]:
#constucting models
chimp = forward_backward(states,obs,observation,trans,obs_trans,bias)
chimpvit = viterbi(states,obs,observation,trans,obs_trans,bias)

In [92]:
#to do calculations in forward more efficient, all betas are computed beforehand
Beta = chimpvit.construct_beta()

In [96]:
#predictions of viterbi algorithm
chimpvit.predict_states(Beta)

[0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1]

In [100]:
#probabilities and predictions of forward backward algorithm
print(chimp.predict_probabilites())
print(chimp.predict_states())

[[5.320410768127444e-11, 3.4050628916015644e-11], [6.129113204882816e-11, 3.83069575305176e-12], [6.129113204882817e-11, 3.830695753051761e-12], [5.320410768127445e-11, 3.405062891601565e-11], [3.1922464608764672e-12, 7.980616152191167e-11], [5.3204107681274444e-11, 3.4050628916015644e-11], [3.1922464608764672e-12, 7.980616152191167e-11], [3.192246460876467e-12, 7.980616152191166e-11], [3.1922464608764664e-12, 7.980616152191167e-11], [5.320410768127444e-11, 3.4050628916015644e-11], [3.1922464608764664e-12, 7.980616152191166e-11], [3.1922464608764664e-12, 7.980616152191164e-11], [3.1922464608764664e-12, 7.980616152191164e-11], [3.192246460876467e-12, 7.980616152191166e-11], [3.192246460876466e-12, 7.980616152191166e-11], [5.320410768127443e-11, 3.405062891601564e-11], [5.3204107681274425e-11, 3.405062891601564e-11], [6.129113204882813e-11, 3.830695753051758e-12], [6.129113204882813e-11, 3.830695753051758e-12], [3.1922464608764656e-12, 7.980616152191164e-11]]
[0, 0, 0, 0, 1, 0, 1, 1, 1, 

The predictions of the chimpanzees prediction are the same for both models